## MySQL_Prepare_ngrok

### Ngrok Setup Steps
1. install Mysql
2. brew install ngrok
3. ngrok config add-authtoken <token>
    * token can be found on ngrok website, needs account
4.  `vim /opt/homebrew/etc/my.cnf` (for mac)
    * change to `bind-address = 0.0.0.0`
5. `brew services restart mysql`
6. go to ngrok website and add a credit card for verification
7. add a mysql user
```mysql
CREATE USER 'newuser'@'%' IDENTIFIED BY 'password';
GRANT ALL PRIVILEGES ON *.* TO 'newuser'@'%';
FLUSH PRIVILEGES;
```
8. restart mysql, and ngrok
9. ngrok tcp 3306
10. use navicat or sth connect to db
    1.  sample host: 6.tcp.ngrok.io
    2. sample port: 10914 (not 3306!!!)



### Environment Setup

1. install Python Packages



In [ ]:
pip install pymysql sqlalchemy faker pandas

2. Connect to DB & Create Tables
    1. Will delete original DB before creating

In [2]:
# Connect to DB -----------------------------------------------------------------------
from sqlalchemy import create_engine, text
from faker import Faker
import pandas as pd

faker = Faker()

# 连接信息
username = ''
password = ''
host = ''
port = ''
database = ''

# 先连接到 mysql 系统库（因为要Drop/Create数据库）
engine_root = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/mysql",
    connect_args={"charset": "utf8mb4"}
)

# Drop + Create Database
with engine_root.connect() as conn:
    conn.execute(text(f"DROP DATABASE IF EXISTS {database};"))
    conn.execute(text(f"""
        CREATE DATABASE {database}
        DEFAULT CHARACTER SET utf8mb4
        DEFAULT COLLATE utf8mb4_unicode_ci;
    """))

print(f"✅ Database `{database}` dropped and recreated successfully!")

# 然后连接到新的 Flight 数据库
engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}",
    connect_args={"charset": "utf8mb4"}
)

# Create Tables -----------------------------------------------------------------------
# 定义所有建表 SQL
tables = [
    # 1. Airlines
    """
    CREATE TABLE IF NOT EXISTS airlines (
        airline_id INT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        alias VARCHAR(255),
        iata VARCHAR(10),
        icao VARCHAR(10),
        country VARCHAR(100),
        active CHAR(1) CHECK (active IN ('Y', 'N'))
    );
    """,

    # 2. Airports
    """
    CREATE TABLE IF NOT EXISTS airports (
        airport_id INT PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        city VARCHAR(255),
        country VARCHAR(255),
        iata VARCHAR(10),
        icao VARCHAR(10),
        latitude FLOAT,
        longitude FLOAT,
        altitude INT,
        timezone FLOAT,
        dst CHAR(1),
        tz_database_time_zone VARCHAR(100)
    );
    """,

    # 3. Aircrafts
    """
    CREATE TABLE IF NOT EXISTS aircrafts (
        aircraft_id INT PRIMARY KEY,
        model VARCHAR(255) NOT NULL,
        manufacturer VARCHAR(255),
        iata_code VARCHAR(10),
        icao_code VARCHAR(10),
        seats INT
    );
    """,

    # 4. Routes
    """
    CREATE TABLE IF NOT EXISTS routes (
        route_id INT PRIMARY KEY AUTO_INCREMENT,
        airline_id INT,
        source_airport_id INT,
        destination_airport_id INT,
        codeshare CHAR(1),
        stops INT,
        equipment VARCHAR(50),
        FOREIGN KEY (airline_id) REFERENCES airlines(airline_id),
        FOREIGN KEY (source_airport_id) REFERENCES airports(airport_id),
        FOREIGN KEY (destination_airport_id) REFERENCES airports(airport_id)
    );
    """,

    # 5. Flights
    """
    CREATE TABLE IF NOT EXISTS flights (
        flight_id INT PRIMARY KEY AUTO_INCREMENT,
        route_id INT,
        flight_number VARCHAR(20),
        departure_time TIME,
        arrival_time TIME,
        flight_date DATE,
        aircraft_id INT,
        status VARCHAR(50),
        FOREIGN KEY (route_id) REFERENCES routes(route_id),
        FOREIGN KEY (aircraft_id) REFERENCES aircrafts(aircraft_id)
    );
    """,

    # 6. Passengers
    """
    CREATE TABLE IF NOT EXISTS passengers (
        passenger_id INT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        passport_number VARCHAR(50) UNIQUE,
        nationality VARCHAR(100),
        date_of_birth DATE
    );
    """,

    # 7. Bookings
    """
    CREATE TABLE IF NOT EXISTS bookings (
        booking_id INT PRIMARY KEY AUTO_INCREMENT,
        passenger_id INT,
        flight_id INT,
        booking_date DATE,
        seat_number VARCHAR(10),
        ticket_price DECIMAL(10,2),
        FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id),
        FOREIGN KEY (flight_id) REFERENCES flights(flight_id)
    );
    """
]

# 循环建表
with engine.connect() as conn:
    for sql in tables:
        conn.execute(text(sql))

print("✅ All tables created successfully!")

✅ Database `Flight` dropped and recreated successfully!
✅ All tables created successfully!


3. Insert Fake Data

In [3]:
# 1. Airlines
airlines = []
for i in range(1, 31):
    airlines.append({
        'airline_id': i,
        'name': faker.company(),
        'alias': faker.word(),
        'iata': faker.lexify(text='??').upper(),
        'icao': faker.lexify(text='???').upper(),
        'country': faker.country(),
        'active': faker.random_element(elements=('Y', 'N'))
    })

airlines_df = pd.DataFrame(airlines)
airlines_df.to_sql('airlines', con=engine, if_exists='append', index=False)

# 2. Airports
airports = []
for i in range(1, 31):
    airports.append({
        'airport_id': i,
        'name': faker.city() + " International Airport",
        'city': faker.city(),
        'country': faker.country(),
        'iata': faker.lexify(text='???').upper(),
        'icao': faker.lexify(text='????').upper(),
        'latitude': faker.latitude(),
        'longitude': faker.longitude(),
        'altitude': faker.random_int(min=0, max=5000),
        'timezone': faker.random_int(min=-12, max=12),
        'dst': faker.random_element(elements=('E', 'A', 'S', 'O', 'Z', 'N', 'U')),
        'tz_database_time_zone': faker.timezone()
    })

airports_df = pd.DataFrame(airports)
airports_df.to_sql('airports', con=engine, if_exists='append', index=False)

# 3. Aircrafts
aircrafts = []
for i in range(1, 31):
    aircrafts.append({
        'aircraft_id': i,
        'model': faker.word().capitalize() + " " + faker.random_element(elements=['737', '777', 'A320', 'A350']),
        'manufacturer': faker.company(),
        'iata_code': faker.lexify(text='??').upper(),
        'icao_code': faker.lexify(text='???').upper(),
        'seats': faker.random_int(min=50, max=400)
    })

aircrafts_df = pd.DataFrame(aircrafts)
aircrafts_df.to_sql('aircrafts', con=engine, if_exists='append', index=False)

# 4. Routes
airlines_ids = pd.read_sql("SELECT airline_id FROM airlines", con=engine)['airline_id'].tolist()
airports_ids = pd.read_sql("SELECT airport_id FROM airports", con=engine)['airport_id'].tolist()

routes = []
for i in range(1, 31):
    source, destination = faker.random_elements(elements=airports_ids, length=2, unique=True)
    routes.append({
        'route_id': i,
        'airline_id': faker.random_element(elements=airlines_ids),
        'source_airport_id': source,
        'destination_airport_id': destination,
        'codeshare': faker.random_element(elements=('Y', 'N')),
        'stops': faker.random_int(min=0, max=2),
        'equipment': faker.lexify(text='???')
    })

routes_df = pd.DataFrame(routes)
routes_df.to_sql('routes', con=engine, if_exists='append', index=False)

# 5. Flights
route_ids = pd.read_sql("SELECT route_id FROM routes", con=engine)['route_id'].tolist()
aircraft_ids = pd.read_sql("SELECT aircraft_id FROM aircrafts", con=engine)['aircraft_id'].tolist()

flights = []
for i in range(1, 61):
    flights.append({
        'flight_id': i,
        'route_id': faker.random_element(elements=route_ids),
        'flight_number': faker.bothify(text='??###').upper(),
        'departure_time': faker.time(),
        'arrival_time': faker.time(),
        'flight_date': faker.date_this_year(),
        'aircraft_id': faker.random_element(elements=aircraft_ids),
        'status': faker.random_element(elements=['Scheduled', 'Cancelled', 'Delayed', 'Departed'])
    })

flights_df = pd.DataFrame(flights)
flights_df.to_sql('flights', con=engine, if_exists='append', index=False)

# 6. Passengers
passengers = []
for i in range(1, 91):
    passengers.append({
        'passenger_id': i,
        'first_name': faker.first_name(),
        'last_name': faker.last_name(),
        'passport_number': faker.unique.bothify(text='??######'),
        'nationality': faker.country(),
        'date_of_birth': faker.date_of_birth(minimum_age=18, maximum_age=80)
    })

passengers_df = pd.DataFrame(passengers)
passengers_df.to_sql('passengers', con=engine, if_exists='append', index=False)

# 7. Bookings
passenger_ids = pd.read_sql("SELECT passenger_id FROM passengers", con=engine)['passenger_id'].tolist()
flight_ids = pd.read_sql("SELECT flight_id FROM flights", con=engine)['flight_id'].tolist()

bookings = []
for i in range(1, 121):
    bookings.append({
        'booking_id': i,
        'passenger_id': faker.random_element(elements=passenger_ids),
        'flight_id': faker.random_element(elements=flight_ids),
        'booking_date': faker.date_this_year(),
        'seat_number': faker.random_element(elements=['A', 'B', 'C', 'D', 'E', 'F']) + str(faker.random_int(min=1, max=30)),
        'ticket_price': faker.pydecimal(left_digits=3, right_digits=2, positive=True)
    })

bookings_df = pd.DataFrame(bookings)
bookings_df.to_sql('bookings', con=engine, if_exists='append', index=False)

120

4. Query Example

In [33]:
import pandas as pd

# 查询航线对应的航班数量
query = """
SELECT nationality, COUNT(*) AS count
FROM passengers
GROUP BY nationality
ORDER BY count DESC
LIMIT 10;
"""

# 用 pandas.read_sql
df = pd.read_sql(query, con=engine)
df.head(100)

,nationality,count
0,Algeria,3
1,Italy,3
2,Saudi Arabia,3
3,Rwanda,3
4,Togo,3
5,Ecuador,2
6,Zambia,2
7,Congo,2
8,Brunei Darussalam,2
9,Turks and Caicos Islands,2
